In [25]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats as st
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.libqsturng import psturng, qsturng
from sklearn.linear_model import LinearRegression


def std_sample_mean(s_population, n):
    """For a sample of size n, calculate the standard deviation of the sample mean,
    given the standard deviation of the population.
    """
    return s_population / np.sqrt(n)


def ci(mean, std, confidence):
    '''Calculate the confidence interval for the specified normal distribution of N(mean, std)
    at given two-sided confidence level.
    '''
    two_sided_confidence = confidence + (1 - confidence) / 2
    std_error = st.norm.ppf(two_sided_confidence)
    return mean - std_error * std, mean + std_error * std


def ci_t(mean, std, df, confidence):
    '''Calculate the confidence interval for the specified t distribution of N(mean, std)
    at given two-sided confidence level.
    '''
    two_sided_confidence = confidence + (1 - confidence) / 2
    std_error = st.t.ppf(two_sided_confidence, df)
    return mean - std_error * std, mean + std_error * std


def r2(t, df):
    """Return the coefficient of determination given the t-statistic of a t-test and the
    degrees of freedom df.
    """
    return t**2 / (t**2 + df)


def t_statistic(r, N):
    """Calculate the t statistic for the specified Pearson's correlation coefficient r and sample size N."""
    degrees_of_freedom = N - 2
    t = r * np.sqrt(degrees_of_freedom / (1 - r**2))
    return t


def p_value(r, N):
    """Calculate the p value for the specified Pearson's correlation coefficient r, sample size N and
    two-tailed confidence = 0.95."""
    degrees_of_freedom = N - 2
    t = abs(t_statistic(r, N))
    return (1 - st.t.cdf(t, degrees_of_freedom)) * 2


# Lesson 18 - Linear regression

In [6]:
filename = "../data/flight_costs.csv"
flights = pd.read_csv(filename).values

flights

array([[ 337.  ,   59.5 ],
       [2565.  ,  509.5 ],
       [ 967.  ,  124.5 ],
       [5124.  , 1480.4 ],
       [2398.  ,  696.23],
       [2586.  ,  559.5 ],
       [7412.  , 1481.5 ],
       [ 522.  ,  474.5 ],
       [1499.  ,  737.5 ]])

In [16]:
x, y = flights[:, 0], flights[:, 1]

r, p = st.pearsonr(x, y)

print(r)

0.9090036493537197


In [17]:
sx = x.std(ddof=1)
sy = y.std(ddof=1)
print(sx, sy)

2315.336824548668 508.18700228798104


In [19]:
b = r * (sy / sx)

print(b)

0.1995147465094843


In [20]:
mx = x.mean()
my = y.mean()
print(mx, my)

2601.1111111111113 680.3477777777778


In [21]:
a = my - b*mx
print(a)

161.38775380144136


In [22]:
print(a + b * 4000)

959.4467398393786


In [23]:
print((500 - a) / b)

1697.179041261805


# Alternative 1 - `sklearn.linear_model.LinearRegression`

In [42]:
reg = LinearRegression(fit_intercept=True, normalize=False, copy_X=True)

X = x.reshape(-1, 1)
Y = y.reshape(-1, 1)
reg.fit(X, Y)

print(reg.coef_)
print(reg.intercept_)

[[0.19951475]]
[161.3877538]


# Alternative 2 - `statmodels.api.OLS`

In [43]:
import statsmodels.api as sm


# Fit and summarize OLS model
X = sm.add_constant(x.reshape(-1, 1))
Y = y.reshape(-1, 1)

model = sm.OLS(Y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     33.30
Date:                Mon, 14 Dec 2020   Prob (F-statistic):           0.000684
Time:                        21:34:28   Log-Likelihood:                -60.441
No. Observations:                   9   AIC:                             124.9
Df Residuals:                       7   BIC:                             125.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        161.3878    117.411      1.375      0.2

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


# Problem Set 15

In [49]:
filename = "../data/vision.csv"
vision = pd.read_csv(filename).values

vision

array([[116,  60],
       [117,  67],
       [120,  64],
       [  1,   8],
       [ 52,  13],
       [ 79,  63],
       [109,  63],
       [ 27,   2],
       [ 85,  46],
       [ 51,  27],
       [ 78,  43],
       [ 55,  24],
       [ 26,  10],
       [ 39,  28],
       [107,  56]], dtype=int64)

In [50]:
x = vision[:, 0].reshape(-1, 1)
y = vision[:, 1].reshape(-1, 1)

In [51]:
# Fit and summarize OLS model
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.863
Method:                 Least Squares   F-statistic:                     89.54
Date:                Mon, 14 Dec 2020   Prob (F-statistic):           3.40e-07
Time:                        21:50:49   Log-Likelihood:                -52.586
No. Observations:                  15   AIC:                             109.2
Df Residuals:                      13   BIC:                             110.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2487      4.830     -0.466      0.6

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))
